In [2]:
import os
import numpy as np
import opensmile
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import classification_report
import optuna

# ------------------- Load SAVEE -------------------
def load_savee_audio(dataset_path):
    audio_files = []
    labels = []

    label_map = {
        'a': 'angry', 'd': 'disgust', 'f': 'fear', 'h': 'happy',
        'n': 'neutral', 'sa': 'sad', 'su': 'surprise'
    }

    for file in os.listdir(dataset_path):
        if file.endswith(".wav"):
            parts = file.split('_')
            emotion_code = parts[1][:2] if parts[1][:2] in label_map else parts[1][0]
            if emotion_code in label_map:
                emotion = label_map[emotion_code]
                audio_files.append(os.path.join(dataset_path, file))
                labels.append(emotion)

    print(f"Loaded {len(audio_files)} audio files from SAVEE.")
    return audio_files, labels

# OpenSMILE Feature Extraction
def extract_opensmile_features(file_path):
    smile = opensmile.Smile(
        feature_set=opensmile.FeatureSet.ComParE_2016,
        feature_level=opensmile.FeatureLevel.Functionals
    )
    features = smile.process_file(file_path)
    return features.values.flatten()

# Dataset path
dataset_path = "C:/Users/samhi/OneDrive/문서/College/s6/Speech Processing/Endsem/archive/ALL"

audio_files, labels = load_savee_audio(dataset_path)

X, y_clean = [], []
for file, label in tqdm(zip(audio_files, labels), total=len(audio_files)):
    try:
        features = extract_opensmile_features(file)
        if features.size > 0:
            X.append(features)
            y_clean.append(label)
    except Exception as e:
        print(f"Error processing {file}: {e}")

X = np.array(X)
y = LabelEncoder().fit_transform(y_clean)
X = StandardScaler().fit_transform(X)

# Train/Test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# ------------------- DCNN with Optuna -------------------
def objective(trial):
    model = Sequential([
        Conv1D(trial.suggest_int('filters1', 32, 128), kernel_size=trial.suggest_int('kernel1', 3, 7),
               activation='relu', input_shape=(X_train.shape[1], 1)),
        MaxPooling1D(pool_size=2),
        Dropout(trial.suggest_float('dropout', 0.2, 0.5)),
        Flatten(),
        Dense(128, activation='relu'),
        Dropout(trial.suggest_float('dropout', 0.2, 0.5)),
        Dense(len(set(y)), activation='softmax')
    ])
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    model.fit(X_train.reshape(-1, X_train.shape[1], 1), y_train, epochs=20, batch_size=trial.suggest_int('batch', 16, 64), verbose=0,
              validation_data=(X_test.reshape(-1, X_test.shape[1], 1), y_test), callbacks=[EarlyStopping(patience=3)])
    _, acc = model.evaluate(X_test.reshape(-1, X_test.shape[1], 1), y_test, verbose=0)
    return acc

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=20)

# Train final model
params = study.best_params
model = Sequential([
    Conv1D(params['filters1'], kernel_size=params['kernel1'], activation='relu', input_shape=(X_train.shape[1], 1)),
    MaxPooling1D(pool_size=2),
    Dropout(params['dropout']),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(params['dropout']),
    Dense(len(set(y)), activation='softmax')
])
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(X_train.reshape(-1, X_train.shape[1], 1), y_train, epochs=50, batch_size=params['batch'], validation_data=(X_test.reshape(-1, X_test.shape[1], 1), y_test), callbacks=[EarlyStopping(patience=5)])

# Evaluation
y_pred = model.predict(X_test.reshape(-1, X_test.shape[1], 1))
y_pred = np.argmax(y_pred, axis=1)

print(classification_report(y_test, y_pred))


Loaded 480 audio files from SAVEE.


100%|██████████| 480/480 [02:29<00:00,  3.20it/s]
[I 2025-04-06 12:30:51,685] A new study created in memory with name: no-name-701314a1-c214-4592-b0d0-86e595c58713


[I 2025-04-06 12:31:54,537] Trial 0 finished with value: 0.6145833134651184 and parameters: {'filters1': 82, 'kernel1': 3, 'dropout': 0.344226840746249, 'batch': 55}. Best is trial 0 with value: 0.6145833134651184.
[I 2025-04-06 12:32:27,829] Trial 1 finished with value: 0.5625 and parameters: {'filters1': 82, 'kernel1': 5, 'dropout': 0.43349568355571366, 'batch': 56}. Best is trial 0 with value: 0.6145833134651184.
[I 2025-04-06 12:33:48,732] Trial 2 finished with value: 0.65625 and parameters: {'filters1': 100, 'kernel1': 3, 'dropout': 0.32632555742281666, 'batch': 30}. Best is trial 2 with value: 0.65625.
[I 2025-04-06 12:34:33,899] Trial 3 finished with value: 0.6979166865348816 and parameters: {'filters1': 44, 'kernel1': 3, 'dropout': 0.29444693026146634, 'batch': 60}. Best is trial 3 with value: 0.6979166865348816.
[I 2025-04-06 12:36:57,577] Trial 4 finished with value: 0.6458333134651184 and parameters: {'filters1': 117, 'kernel1': 7, 'dropout': 0.3897025241134324, 'batch': 50}

Epoch 1/50
24/24 [==============================] - 6s 239ms/step - loss: 6.1621 - accuracy: 0.3073 - val_loss: 1.4376 - val_accuracy: 0.4792
Epoch 2/50
24/24 [==============================] - 6s 235ms/step - loss: 1.2175 - accuracy: 0.5052 - val_loss: 1.3017 - val_accuracy: 0.5312
Epoch 3/50
24/24 [==============================] - 6s 233ms/step - loss: 0.9445 - accuracy: 0.6276 - val_loss: 1.3347 - val_accuracy: 0.5938
Epoch 4/50
24/24 [==============================] - 6s 236ms/step - loss: 0.7887 - accuracy: 0.6979 - val_loss: 1.2179 - val_accuracy: 0.6354
Epoch 5/50
24/24 [==============================] - 6s 232ms/step - loss: 0.5955 - accuracy: 0.7708 - val_loss: 1.3014 - val_accuracy: 0.6042
Epoch 6/50
24/24 [==============================] - 6s 233ms/step - loss: 0.5183 - accuracy: 0.8073 - val_loss: 1.1275 - val_accuracy: 0.6250
Epoch 7/50
24/24 [==============================] - 6s 235ms/step - loss: 0.3570 - accuracy: 0.8906 - val_loss: 1.1830 - val_accuracy: 0.6667
Epoch 